In [50]:
import json
import pyodbc
import hashlib
from PIL import Image, ImageTk
from typing import Dict, List, Tuple
import customtkinter as ctk
import os
import webbrowser

In [51]:
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

In [52]:
def create_connection():
    try:
        connection_string = (
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=LAPTOP-MCTQMF1J\\SQLEXPRESS01;"
            "DATABASE=Cinema;"
            "Trusted_Connection=yes;"
        )
        connection = pyodbc.connect(connection_string)
        print("Connection successful.")
        return connection
    except pyodbc.Error as e:
        print("Error connecting to the database:", e)
        return None

In [53]:
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

In [54]:
def register_user(name, password, phone_no):
    conn = create_connection()
    if not conn:
        return False, "Database connection failed"
    
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Customer WHERE Phone_No = ?", phone_no)
    if cursor.fetchone():
        conn.close()
        return False, "Phone number already registered"
    
    hashed_password = hash_password(password)
    try:
        cursor.execute(
            "INSERT INTO Customer (Name, Password, Phone_No, IsAdministrator) VALUES (?, ?, ?, 0)",
            name, hashed_password, phone_no
        )
        conn.commit()
        conn.close()
        return True, "User registered successfully!"
    except Exception as e:
        conn.close()
        return False, f"Registration failed: {str(e)}"

In [55]:
def authenticate_user(username, password):
    conn = create_connection()
    if not conn:
        return None, None
    
    cursor = conn.cursor()
    hashed_password = hash_password(password.strip())
    cursor.execute(
        "SELECT Name, IsAdministrator, User_ID FROM Customer WHERE LOWER(TRIM(Name))= ? AND Password = ?",
        (username.strip().lower(), hashed_password)
    )
    user = cursor.fetchone()
    conn.close()
    
    if user:
        role = "admin" if user[1] == 1 else "user"
        return f"Welcome, {user[0]}!", role, user[2]
    return "Invalid username or password.", None, None


In [56]:
def fetch_show_times(show_id):
    conn = create_connection()
    cursor = conn.cursor()
    query = """
        SELECT CS.Show_ID, CS.Show_Time, tr.Type, CS.Seat_Matrix 
        FROM Cinema_Shows CS
        JOIN Theatre_Rooms tr ON CS.Room_ID = tr.Room_ID 
        WHERE CS.Show_ID = ?
    """
    cursor.execute(query, show_id)
    result = cursor.fetchall()
    conn.close()
    return result if result else []

In [57]:

def book_ticket(user_id, show_id, seat_no):
    conn = create_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute("""
            SELECT CS.Seat_Matrix, TR.Price_Per_Seat ,CS.Movie_ID
            FROM Cinema_Shows CS 
            JOIN Theatre_Rooms TR ON CS.Room_ID = TR.Room_ID 
            WHERE CS.Show_ID = ?
        """, show_id)
        result = cursor.fetchone()
        
        if not result:
            conn.close()
            return False, "Invalid Show ID."
        
        seat_matrix = json.loads(result[0])
        price_per_seat = result[1]
        movie_id =result[2]
        
        if seat_no in seat_matrix and seat_matrix[seat_no] == "Available":
            seat_matrix[seat_no] = "Booked"
            updated_matrix = json.dumps(seat_matrix)
            
            cursor.execute("""
                UPDATE Cinema_Shows 
                SET Seat_Matrix = ?, Available_Seats = Available_Seats - 1 
                WHERE Show_ID = ?
            """, (updated_matrix, show_id))
            
            cursor.execute("""
                INSERT INTO Booking (User_ID, Show_ID, Movie_ID, Seat_No, Total_Amount) 
                VALUES (?, ?, ?, ?,?)
            """, (user_id, show_id,movie_id, seat_no, price_per_seat))
            
            conn.commit()
            conn.close()
            return True, f"Booking successful! Seat {seat_no} has been booked. Total amount: ${price_per_seat:.2f}"
        else:
            conn.close()
            return False, "Seat is already booked or invalid."
            
    except Exception as e:
        conn.close()
        return False, f"Booking failed: {str(e)}"

In [58]:

def cancel_booking(booking_id):
    conn = create_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute("""
            SELECT Show_ID, Seat_No FROM Booking WHERE Booking_ID = ?
        """, booking_id)
        booking = cursor.fetchone()
        
        if not booking:
            conn.close()
            return False, "Invalid Booking ID."
        
        show_id, seat_no = booking
        cursor.execute("SELECT Seat_Matrix FROM Cinema_Shows WHERE Show_ID = ?", show_id)
        result = cursor.fetchone()
        
        if not result:
            conn.close()
            return False, "Invalid Show ID."
        
        seat_matrix = json.loads(result[0])
        if seat_no in seat_matrix and seat_matrix[seat_no] == "Booked":
            seat_matrix[seat_no] = "Available"
            updated_matrix = json.dumps(seat_matrix)
            
            cursor.execute("""
                UPDATE Cinema_Shows 
                SET Seat_Matrix = ?, Available_Seats = Available_Seats + 1 
                WHERE Show_ID = ?
            """, (updated_matrix, show_id))
            
            cursor.execute("DELETE FROM Booking WHERE Booking_ID = ?", booking_id)
            conn.commit()
            conn.close()
            return True, f"Booking cancelled successfully! Seat {seat_no} is now available."
        else:
            conn.close()
            return False, "Seat is already available or invalid."
            
    except Exception as e:
        conn.close()
        return False, f"Cancellation failed: {str(e)}"

In [59]:
class ScrollableFrame(ctk.CTkScrollableFrame):
    def _init_(self, container, *args, **kwargs):
        super()._init_(container, *args, **kwargs)

In [60]:
class App(ctk.CTk):
    def __init__(self):
        super().__init__()
        
        self.title("Cinema Management System")
        self.geometry("1200x800")
        
        # Movie data
        self.movies = [
                {"id": 1, "show_id": 6, "title": "The Shawshank Redemption", "image": r"C:\Users\20101\Downloads\Shawshank_Redemption.jpg"},
                {"id": 2, "show_id": 7, "title": "The Godfather", "image": r"C:\Users\20101\Downloads\The_Godfather.jpg"},
                {"id": 3, "show_id": 8, "title": "The Dark Knight", "image": r"C:\Users\20101\Downloads\The_Dark_Knight.jpg"},
                {"id": 4, "show_id": 9, "title": "Inception", "image": r"C:\Users\20101\Downloads\Inception.jpg"},
                {"id": 5, "show_id": 10, "title": "Forrest Gump", "image": r"C:\Users\20101\Downloads\Forrest_Gump.jpg"},
                {"id": 6, "show_id": 11, "title": "The Lord of the Rings", "image": r"C:\Users\20101\Downloads\Lord_of_the_rings.jpg"},
                {"id": 7, "show_id": 12, "title": "Pulp Fiction", "image": r"C:\Users\20101\Downloads\pulp fiction.jpg"},
                {"id": 8, "show_id": 13, "title": "Interstellar", "image": r"C:\Users\20101\Downloads\interstellar.jpg"}
            ]

        
        self.current_user_id = None
        self.current_user_role = None
        
        # Initialize with login page
        self.show_login_page()
    
    def clear_window(self):
        for widget in self.winfo_children():
            widget.destroy()
    
    def show_login_page(self):
        self.clear_window()
        
        frame = ctk.CTkFrame(self)
        frame.pack(pady=20, padx=60, fill="both", expand=True)
        
        label = ctk.CTkLabel(frame, text="Cinema Management System", font=("Roboto", 24))
        label.pack(pady=12, padx=10)
        
        # Login widgets
        self.username_entry = ctk.CTkEntry(frame, placeholder_text="Username")
        self.username_entry.pack(pady=12, padx=10)
        
        self.password_entry = ctk.CTkEntry(frame, placeholder_text="Password", show="*")
        self.password_entry.pack(pady=12, padx=10)
        
        login_button = ctk.CTkButton(frame, text="Login", command=self.login)
        login_button.pack(pady=12, padx=10)
        
        register_button = ctk.CTkButton(frame, text="Register New Account", command=self.show_register_page)
        register_button.pack(pady=12, padx=10)
    
    def show_register_page(self):
        self.clear_window()
        
        frame = ctk.CTkFrame(self)
        frame.pack(pady=20, padx=60, fill="both", expand=True)
        
        label = ctk.CTkLabel(frame, text="Register New Account", font=("Roboto", 24))
        label.pack(pady=12, padx=10)
        
        self.reg_name_entry = ctk.CTkEntry(frame, placeholder_text="Name")
        self.reg_name_entry.pack(pady=12, padx=10)
        
        self.reg_password_entry = ctk.CTkEntry(frame, placeholder_text="Password", show="*")
        self.reg_password_entry.pack(pady=12, padx=10)
        
        self.reg_phone_entry = ctk.CTkEntry(frame, placeholder_text="Phone Number")
        self.reg_phone_entry.pack(pady=12, padx=10)
        
        register_button = ctk.CTkButton(frame, text="Register", command=self.register)
        register_button.pack(pady=12, padx=10)
        
        back_button = ctk.CTkButton(frame, text="Back to Login", command=self.show_login_page)
        back_button.pack(pady=12, padx=10)
    
    def login(self):
        username = self.username_entry.get().strip()
        password = self.password_entry.get().strip()
        
        # Clear any existing error messages
        for widget in self.winfo_children():
            if isinstance(widget, ctk.CTkLabel) and widget.cget("text_color") == "red":
                widget.destroy()
        
        # Validate inputs
        if not username or not password:
            error_label = ctk.CTkLabel(self, text="Username and password are required", text_color="red")
            error_label.pack(pady=12, padx=10)
            return
        
        message, role, user_id = authenticate_user(username, password)
        if role:
            self.current_user_id = user_id
            self.current_user_role = role
            if role == "admin":
                self.show_admin_page()
            else:
                self.show_user_page()
        else:
            # Show error message within the login frame
            error_label = ctk.CTkLabel(self.winfo_children()[0], text=message, text_color="red")
            error_label.pack(pady=12, padx=10)
    
    def register(self):
        name = self.reg_name_entry.get().strip()
        password = self.reg_password_entry.get().strip()
        phone = self.reg_phone_entry.get().strip()
        
        # Clear any existing error messages
        for widget in self.winfo_children():
            if isinstance(widget, ctk.CTkLabel) and widget.cget("text_color") == "red":
                widget.destroy()
        
        # Validate inputs
        if not name or not password or not phone:
            error_label = ctk.CTkLabel(self, text="All fields are required", text_color="red")
            error_label.pack(pady=12, padx=10)
            return
        
        # Validate phone number
        if not phone.isdigit() or len(phone) != 11:
            error_label = ctk.CTkLabel(self, text="Phone number must be 11 digits", text_color="red")
            error_label.pack(pady=12, padx=10)
            return
        
        success, message = register_user(name, password, phone)
        if success:
            self.show_login_page()
        else:
            # Show error message within the registration frame
            error_label = ctk.CTkLabel(self.winfo_children()[0], text=message, text_color="red")
            error_label.pack(pady=12, padx=10)
    def show_user_page(self):
        self.clear_window()
        
        # Create main scrollable container
        container = ScrollableFrame(self, width=1150, height=750)
        container.pack(pady=20, padx=20, fill="both", expand=True)
        
        # Snacks section
        snacks_label = ctk.CTkLabel(container, text="Available Snacks", font=("Roboto", 20))
        snacks_label.pack(pady=(12, 20))
        
        snacks_frame = ctk.CTkFrame(container)
        snacks_frame.pack(fill="x", padx=20, pady=10)
        
        # Sample snacks data with image paths
        snacks = [
            {"name": "Popcorn", "price": "5.99", "image": r"C:\Users\20101\Downloads\popcorn.jpg"},
            {"name": "Soda", "price": "2.99", "image": r"C:\Users\20101\Downloads\V_cola.jpg"},
            {"name": "Nachos", "price": "4.99", "image": r"C:\Users\20101\Downloads\nachos .jpg"},
            {"name": "Candy", "price": "3.99", "image": r"C:\Users\20101\Downloads\Jelly_beans.jpg"},
            {"name": "Hot Dog", "price": "4.99", "image": r"C:\Users\20101\Downloads\hotdog.jpg"}
        ]
        
        # Display snacks in a grid
        for i, snack in enumerate(snacks):
            frame = ctk.CTkFrame(snacks_frame)
            frame.grid(row=0, column=i, padx=10, pady=10)
            
            # Load snack image
            snack_image = self.load_image(
                snack["image"], 
                size=(100, 100)
            )
            if snack_image:
                image_label = ctk.CTkLabel(frame, image=snack_image, text="")
                image_label.image = snack_image  # Keep a reference
            else:
                image_label = ctk.CTkLabel(frame, text="[Snack Image]", width=100, height=100)
            image_label.pack(pady=5)
            
            name_label = ctk.CTkLabel(frame, text=snack["name"])
            name_label.pack()
            
            price_label = ctk.CTkLabel(frame, text=f"${snack['price']}")
            price_label.pack()
        
        # Movies section
        movies_label = ctk.CTkLabel(container, text="Now Showing", font=("Roboto", 20))
        movies_label.pack(pady=(20, 12))
        
        # Create a frame for each row of movies (4 movies per row)
        for i in range(0, len(self.movies), 4):
            row_frame = ctk.CTkFrame(container)
            row_frame.pack(fill="x", padx=20, pady=10)
            
            # Add movies to this row
            for j, movie in enumerate(self.movies[i:i+4]):
                frame = ctk.CTkFrame(row_frame)
                frame.grid(row=0, column=j, padx=10, pady=10)
                
                # Load movie poster
                movie_image = self.load_image(movie["image"])
                if movie_image:
                    image_label = ctk.CTkLabel(frame, image=movie_image, text="")
                    image_label.image = movie_image  # Keep a reference
                else:
                    image_label = ctk.CTkLabel(frame, text="[Movie Poster]", width=150, height=225)
                image_label.pack(pady=5)
                
                name_label = ctk.CTkLabel(frame, text=movie["title"])
                name_label.pack()
                
                showtimes_button = ctk.CTkButton(
                    frame,
                    text="Show Times",
                    command=lambda m=movie["id"]: self.show_movie_times(m)
                )
                showtimes_button.pack(pady=5)
        
        # Cancel booking section
        cancel_frame = ctk.CTkFrame(container)
        cancel_frame.pack(fill="x", padx=20, pady=20)
        
        cancel_label = ctk.CTkLabel(cancel_frame, text="Cancel Booking", font=("Roboto", 16))
        cancel_label.pack(pady=5)
        
        booking_id_entry = ctk.CTkEntry(cancel_frame, placeholder_text="Enter Booking ID")
        booking_id_entry.pack(pady=5)
        
        cancel_button = ctk.CTkButton(
            cancel_frame,
            text="Cancel Booking",
            command=lambda: self.cancel_user_booking(booking_id_entry.get())
        )
        cancel_button.pack(pady=5)
        
        # Logout button
        logout_button = ctk.CTkButton(container, text="Logout", command=self.show_login_page)
        logout_button.pack(pady=20)
    
    def show_admin_page(self):
        self.clear_window()
        
        frame = ctk.CTkFrame(self)
        frame.pack(pady=20, padx=60, fill="both", expand=True)
        
        label = ctk.CTkLabel(frame, text="Admin Dashboard", font=("Roboto", 24))
        label.pack(pady=12, padx=10)
        
        # Add movie form
        title_entry = ctk.CTkEntry(frame, placeholder_text="Movie Title")
        title_entry.pack(pady=12, padx=10)
        
        genre_entry = ctk.CTkEntry(frame, placeholder_text="Genre")
        genre_entry.pack(pady=12, padx=10)
        
        duration_entry = ctk.CTkEntry(frame, placeholder_text="Duration (minutes)")
        duration_entry.pack(pady=12, padx=10)
        
        description_entry = ctk.CTkEntry(frame, placeholder_text="Description")
        description_entry.pack(pady=12, padx=10)
        
        status_entry = ctk.CTkEntry(frame, placeholder_text="Status")
        status_entry.pack(pady=12, padx=10)
        
        image_path_entry = ctk.CTkEntry(frame, placeholder_text= r"Image Path")
        image_path_entry.pack(pady=12, padx=10)
        
        add_button = ctk.CTkButton(frame, text="Add Movie", command=lambda: self.add_movie(
            title_entry.get(),
            genre_entry.get(),
            duration_entry.get(),
            description_entry.get(),
            status_entry.get(),
            image_path_entry.get()
        ))
        add_button.pack(pady=12, padx=10)
        
        logout_button = ctk.CTkButton(frame, text="Logout", command=self.show_login_page)
        logout_button.pack(pady=12, padx=10)

    def add_movie(self, title, genre, duration, description, status, image_path):
        # Add the movie to the database (you need to implement this)
        # For now, we'll just add it to the self.movies list
        new_movie = {
            "id": len(self.movies) + 1,  # Auto-generate ID
            "show_id": len(self.movies) + 6,  # Auto-generate show ID
            "title": title,
            "image": image_path,
        }
        self.movies.append(new_movie)
        
        # Refresh the user page to show the new movie
        if self.current_user_role == "user":
            self.show_user_page()
        elif self.current_user_role == "admin":
            self.show_admin_page()
    def load_image(self, image_path, size=(150, 225)):
        try:
            image = Image.open(image_path)
            image = image.resize(size, Image.LANCZOS)
            return ImageTk.PhotoImage(image)
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return None

    def show_user_page(self):
        self.clear_window()
        
        # Create main scrollable container
        container = ScrollableFrame(self, width=1150, height=750)
        container.pack(pady=20, padx=20, fill="both", expand=True)
        
        # Snacks section
        snacks_label = ctk.CTkLabel(container, text="Available Snacks", font=("Roboto", 20))
        snacks_label.pack(pady=(12, 20))
        
        snacks_frame = ctk.CTkFrame(container)
        snacks_frame.pack(fill="x", padx=20, pady=10)
        
        # Sample snacks data with image paths
        snacks = [
            {"name": "Popcorn", "price": "5.99", "image": r"C:\Users\20101\Downloads\popcorn.jpg"},
            {"name": "Soda", "price": "2.99", "image": r"C:\Users\20101\Downloads\V_cola.jpg"},
            {"name": "Nachos", "price": "4.99", "image": r"C:\Users\20101\Downloads\nachos .jpg"},
            {"name": "Candy", "price": "3.99", "image": r"C:\Users\20101\Downloads\Jelly_beans.jpg"},
            {"name": "Hot Dog", "price": "4.99", "image": r"C:\Users\20101\Downloads\hotdog.jpg"}
        ]
        
        # Display snacks in a grid
        for i, snack in enumerate(snacks):
            frame = ctk.CTkFrame(snacks_frame)
            frame.grid(row=0, column=i, padx=10, pady=10)
            
            # Load snack image
            snack_image = self.load_image(
                snack["image"], 
                size=(100, 100)
            )
            if snack_image:
                image_label = ctk.CTkLabel(frame, image=snack_image, text="")
                image_label.image = snack_image  # Keep a reference
            else:
                image_label = ctk.CTkLabel(frame, text="[Snack Image]", width=100, height=100)
            image_label.pack(pady=5)
            
            name_label = ctk.CTkLabel(frame, text=snack["name"])
            name_label.pack()
            
            price_label = ctk.CTkLabel(frame, text=f"${snack['price']}")
            price_label.pack()
        
        # Movies section
        movies_label = ctk.CTkLabel(container, text="Now Showing", font=("Roboto", 20))
        movies_label.pack(pady=(20, 12))
        
        # Create a frame for each row of movies (4 movies per row)
        for i in range(0, len(self.movies), 4):
            row_frame = ctk.CTkFrame(container)
            row_frame.pack(fill="x", padx=20, pady=10)
            
            # Add movies to this row
            for j, movie in enumerate(self.movies[i:i+4]):
                frame = ctk.CTkFrame(row_frame)
                frame.grid(row=0, column=j, padx=10, pady=10)
                
                # Load movie poster
                movie_image = self.load_image(movie["image"])
                if movie_image:
                    image_label = ctk.CTkLabel(frame, image=movie_image, text="")
                    image_label.image = movie_image  # Keep a reference
                else:
                    image_label = ctk.CTkLabel(frame, text="[Movie Poster]", width=150, height=225)
                image_label.pack(pady=5)
                
                name_label = ctk.CTkLabel(frame, text=movie["title"])
                name_label.pack()
                
                showtimes_button = ctk.CTkButton(
                    frame,
                    text="Show Times",
                    command=lambda m=movie["id"]: self.show_movie_times(m)
                )
                showtimes_button.pack(pady=5)
        
        # Cancel booking section
        cancel_frame = ctk.CTkFrame(container)
        cancel_frame.pack(fill="x", padx=20, pady=20)
        
        cancel_label = ctk.CTkLabel(cancel_frame, text="Cancel Booking", font=("Roboto", 16))
        cancel_label.pack(pady=5)
        
        booking_id_entry = ctk.CTkEntry(cancel_frame, placeholder_text="Enter Booking ID")
        booking_id_entry.pack(pady=5)
        
        cancel_button = ctk.CTkButton(
            cancel_frame,
            text="Cancel Booking",
            command=lambda: self.cancel_user_booking(booking_id_entry.get())
        )
        cancel_button.pack(pady=5)
        
        # Logout button
        logout_button = ctk.CTkButton(container, text="Logout", command=self.show_login_page)
        logout_button.pack(pady=20)
    def show_movie_times(self, movie_id):
        # Find movie from self.movies list
        movie = next((m for m in self.movies if m["id"] == movie_id), None)
        if not movie:
            return
            
        # Create a new window for showtimes
        times_window = ctk.CTkToplevel(self)
        times_window.title(f"Show Times - {movie['title']}")
        times_window.geometry("400x600")
        
        # Make the window scrollable
        scroll_frame = ctk.CTkScrollableFrame(times_window, width=380, height=580)
        scroll_frame.pack(pady=10, padx=10, fill="both", expand=True)
        
        # Fetch showtimes using show_id instead of movie_id
        showtimes = fetch_show_times(movie['show_id'])
        
        if not showtimes:
            label = ctk.CTkLabel(scroll_frame, text="No show times available")
            label.pack(pady=20)
            return
        
        for show in showtimes:
            show_id, time, room_type, seat_matrix = show
            
            frame = ctk.CTkFrame(scroll_frame)
            frame.pack(fill="x", padx=20, pady=10)
            
            time_label = ctk.CTkLabel(frame, text=f"Time: {time}")
            time_label.pack()
            
            room_label = ctk.CTkLabel(frame, text=f"Room Type: {room_type}")
            room_label.pack()
            
            select_button = ctk.CTkButton(
                frame,
                text="Select Seats",
                command=lambda s=show: self.show_seat_selection(s)
            )
            select_button.pack(pady=5)
    def show_seat_selection(self, show_data):
        show_id, time, room_type, seat_matrix_str = show_data
        seat_matrix = json.loads(seat_matrix_str)
        
        # Create a new window for seat selection
        seat_window = ctk.CTkToplevel(self)
        seat_window.title("Seat Selection")
        seat_window.geometry("600x500")
        
        # Add a rectangle representing the screen - tripled width
        seat_window.geometry("800x500")  # Made window wider
    
       # Create a container frame to center the screen
        container_frame = ctk.CTkFrame(seat_window, fg_color="transparent")
        container_frame.pack(expand=True, fill="both", padx=20)
        
        # Add a rectangle representing the screen - tripled width
        screen_frame = ctk.CTkFrame(
            container_frame, 
            height=20, 
            width=600,  # Tripled from original 200
            fg_color="gray"
        )
        screen_frame.pack(pady=20)
        # Use place to force the frame to maintain its size
        screen_frame.pack_propagate(False)
        
        screen_label = ctk.CTkLabel(
            screen_frame, 
            text="SCREEN", 
            font=("Roboto", 14),
            text_color="white"  # Made text white for better visibility on gray
        )
        screen_label.pack(pady=5)
        
        # Create seat grid
        seat_frame = ctk.CTkFrame(seat_window)
        seat_frame.pack(pady=20, padx=20)
        
        # Convert seat matrix to grid layout
        rows = sorted(set(seat[0] for seat in seat_matrix.keys()))
        cols = sorted(set(int(seat[1]) for seat in seat_matrix.keys()))
        
        # Create legend
        legend_frame = ctk.CTkFrame(seat_window)
        legend_frame.pack(pady=10)
        
        available_sample = ctk.CTkButton(legend_frame, text="Available", state="normal", width=20)
        available_sample.grid(row=0, column=0, padx=5)
        
        booked_sample = ctk.CTkButton(legend_frame, text="Booked", state="disabled", width=20)
        booked_sample.grid(row=0, column=1, padx=5)
        
        # Create seat buttons
        for i, row in enumerate(rows):
            for j, col in enumerate(cols):
                seat_id = f"{row}{col}"
                is_available = seat_matrix.get(seat_id) == "Available"
                
                button = ctk.CTkButton(
                    seat_frame,
                    text=seat_id,
                    width=40,
                    height=40,
                    state="normal" if is_available else "disabled",
                    command=lambda s=seat_id: self.book_seat(show_id, s)
                )
                button.grid(row=i, column=j, padx=5, pady=5)
        
    def book_seat(self, show_id, seat_no):
        if not self.current_user_id:
            return
        
        success, message = book_ticket(self.current_user_id, show_id, seat_no)
        
        # Show result in a popup
        popup = ctk.CTkToplevel(self)
        popup.title("Booking Result")
        popup.geometry("400x200")
        
        if success:
            # Fetch the booking ID
            conn = create_connection()
            cursor = conn.cursor()
            cursor.execute("SELECT MAX(Booking_ID) FROM Booking")
            booking_id = cursor.fetchone()[0]
            conn.close()
            
            # Create a frame to hold the message and link
            frame = ctk.CTkFrame(popup)
            frame.pack(pady=20, padx=20, fill="both", expand=True)
            
            # Add booking confirmation message
            message_label = ctk.CTkLabel(frame, text=f"{message}\nBooking ID: {booking_id}")
            message_label.pack(pady=(10, 5))
            
            # Add clickable feedback link
            feedback_link = "https://docs.google.com/forms/d/e/1FAIpQLSfbisqqjEqNDmlDStex6nPZjytDLvKAKVEm8rlHv_LRVaxfsw/viewform?usp=dialog"
            link_label = ctk.CTkLabel(frame, text="Click here to provide feedback", text_color="blue", cursor="hand2")
            link_label.pack(pady=5)
            link_label.bind("<Button-1>", lambda e: webbrowser.open_new(feedback_link))
        else:
            label = ctk.CTkLabel(popup, text=message)
            label.pack(pady=20)
        
        ok_button = ctk.CTkButton(popup, text="OK", command=popup.destroy)
        ok_button.pack(pady=10)
    
    def cancel_user_booking(self, booking_id):
        try:
            booking_id = int(booking_id)
            success, message = cancel_booking(booking_id)
            
            # Show result in a popup
            popup = ctk.CTkToplevel(self)
            popup.title("Cancellation Result")
            popup.geometry("300x150")
            
            label = ctk.CTkLabel(popup, text=message)
            label.pack(pady=20)
            
            ok_button = ctk.CTkButton(popup, text="OK", command=popup.destroy)
            ok_button.pack(pady=10)
            
        except ValueError:
            # Show error for invalid booking ID
            popup = ctk.CTkToplevel(self)
            popup.title("Error")
            popup.geometry("300x150")
            
            label = ctk.CTkLabel(popup, text="Please enter a valid booking ID")
            label.pack(pady=20)
            
            ok_button = ctk.CTkButton(popup, text="OK", command=popup.destroy)
            ok_button.pack(pady=10)


In [61]:
app = App()
app.mainloop()

Connection successful.
Connection successful.
Connection successful.
Connection successful.
Connection successful.
